# Installation of Libraries

If you use our code, please cite:

@misc{2024<br>
  title = {Mistral -7b hosting},<br>
  author = {Hamza Farooq, Darshil Modi, Kanwal Mehreen, Nazila Shafiei},<br>
  keywords = {LLM Hosting, Mistral-7b},<br>
  year = {2024},<br>
  copyright = {MIT, non-exclusive license}<br>
}


In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
!pip install git+https://github.com/huggingface/transformers torch accelerate bitsandbytes langchain

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-o1zua2yl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-o1zua2yl
  Resolved https://github.com/huggingface/transformers to commit 93aafdc620d39b9ec714ffecf015a085ea221282
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.7 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10306367 sha256=39c08d4cf6924d42421e2e509bf00b10149f8549449605d50e41db8c03c28fd9
  Stored in directory: /tmp/pip-ephem-wheel-cache-lrjv8fe1/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Succe

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install pyngrok --quiet
!pip install fastapi nest-asyncio --quiet
!pip install uvicorn --quiet
!pip install langchain-community --quiet
!pip install -U langchain-huggingface --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.4 MB/s eta 0:00:00


# Setup

Get you Hugging Face Token, [here](https://huggingface.co/settings/tokens)

In [ ]:
from google.colab import userdata
HUGGING_FACE_TOKEN = userdata.get('HUGGING_FACE_TOKEN')

In [ ]:
!huggingface-cli login --token $HUGGING_FACE_TOKEN --add-to-git-credential

Token is valid (permission: fineGrained).
The token `maven-course` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `maven-course`


In [ ]:
# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer , BitsAndBytesConfig
import transformers
import torch
from langchain_huggingface import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from threading import Thread

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
!nvidia-smi

Tue Dec 24 06:37:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

# Basic Prompt

**Note:** If you are on windows then bitsandbytes won't work. You can counter this with bitsandbytes-windows but that is and older version that only supports 8-bit quantization.

load_in_4bit=True enables 4-bit quantization, which reduces the model's memory footprint by converting the model weights from their original precision (typically FP16 or FP32) to 4-bit representation.


bnb_4bit_compute_dtype=torch.bfloat16 specifies that the computation during inference should use bfloat16 precision, which provides a good balance between numerical precision and memory efficiency

In [ ]:
# Define the model ID for the desired model
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

# Define the quantization configuration for the model; 4bit for storing and BF16 while inference
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the model using the model ID and quantization configuration
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map='auto')

# Load the tokenizer associated with the model
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
vegetarian_recipe_prompt = """### Instruction: Act as a gourmet chef.
what is the best chicken dish for large groups

 ### Answer:
 """
prompt_template = """
### [INST]
Instruction: Act as a gourmet chef.


### QUESTION:
what is the best chicken dish for large groups

[/INST]
 """

# Encode the instruction using the tokenizer
encoded_instruction = tokenizer(prompt_template, return_tensors="pt", add_special_tokens=True)

# Move the encoded instruction to the appropriate device (e.g., GPU)
model_inputs = encoded_instruction.to(device)

# Generate text based on the model inputs
generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

# Decode the generated text
decoded = tokenizer.batch_decode(generated_ids)

# Print the decoded output
print(decoded)

['<s> \n### [INST]\nInstruction: Act as a gourmet chef.\n\n\n### QUESTION:\nwhat is the best chicken dish for large groups\n\n[/INST]\n 1. Chicken fajitas: a classic dish that can feed a large group with little preparation. Simply cook the chicken with peppers, onions, and other desired vegetables, and serve with tortillas, salsa, guacamole, and sour cream.\n\n2. Chicken kebabs: a family-friendly and flavorful option that is easy to make with skewers and grilled chicken. Add some herbs and spices to the chicken, and other meats like bell peppers and onions for a savory taste.\n\n3. Chicken stew: a comforting and hearty dish that is simple to make in a large pot. You can add carrots, celery, onions, tomatoes, and corn or chickpeas for some extra flavor and nutrition.\n\n4. Chicken curry: a delicious and spicy option that is perfect for a large group gathering. You can customize the recipe with your favorite curry spices and add vegetables like spinach or carrots, and rice for a complete

In [ ]:
response = decoded[0].split("[/INST]")[1]
print((response.split('</s>')[0]).strip())

1. Chicken fajitas: a classic dish that can feed a large group with little preparation. Simply cook the chicken with peppers, onions, and other desired vegetables, and serve with tortillas, salsa, guacamole, and sour cream.

2. Chicken kebabs: a family-friendly and flavorful option that is easy to make with skewers and grilled chicken. Add some herbs and spices to the chicken, and other meats like bell peppers and onions for a savory taste.

3. Chicken stew: a comforting and hearty dish that is simple to make in a large pot. You can add carrots, celery, onions, tomatoes, and corn or chickpeas for some extra flavor and nutrition.

4. Chicken curry: a delicious and spicy option that is perfect for a large group gathering. You can customize the recipe with your favorite curry spices and add vegetables like spinach or carrots, and rice for a complete meal.

5. Grilled chicken salad: a healthy and delicious option that is perfect for a warm summer day. Grill some chicken with your favorite 

# Creating LLM Chain to run a RAG prompt


In [ ]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",  # Specify the task as text generation
    temperature=0.3,  # Temperature parameter for controlling randomness in sampling
    repetition_penalty=1.1,  # Repetition penalty parameter to avoid repeating tokens
    return_full_text=True,  # Flag to return full text instead of a list of generated tokens
    max_new_tokens=1000,  # Maximum number of tokens to generate
    do_sample=True  # Flag to use sampling during text generation
)

prompt_template = """
### [INST]
Instruction: I will ask you a QUESTION and give you a CONTEXT and you will respond with an answer easily understandable.

### CONTEXT:
{context}

### QUESTION:
{question}

[/INST]
 """

# Create HuggingFacePipeline object wrapping the text generation pipeline
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt object from the prompt template with input variables as context and question
prompt = PromptTemplate(
    input_variables=["context", "question"],  # Specify input variables for the prompt
    template=prompt_template,  # Specify the template for the prompt
)

# Create language model chain (llm_chain) with HuggingFacePipeline and prompt
llm_chain = prompt | mistral_llm | StrOutputParser()


Device set to use cuda:0


In [ ]:
prompt_base = """
### [INST]
Instruction: Act as a gourmet chef.


### QUESTION:
what is the best chicken dish for large groups

[/INST]
 """


In [ ]:
response=text_generation_pipeline(prompt_base)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
response

[{'generated_text': '\n### [INST]\nInstruction: Act as a gourmet chef.\n\n\n### QUESTION:\nwhat is the best chicken dish for large groups\n\n[/INST]\n  As a gourmet chef, I would recommend roasting a whole chicken for a large group. This classic dish is both flavorful and visually appealing, and it can easily be served with a variety of sides such as roasted vegetables, mashed potatoes, or a fresh salad. To ensure that the chicken is perfectly cooked, you can use a thermometer to check the internal temperature of the bird. The thickest part of the thigh should reach an internal temperature of 165°F (74°C) before removing it from the oven. You can also add some herbs and spices to the chicken before roasting it to give it an extra boost of flavor.'}]

In [ ]:
r=response[0].get('generated_text').split("[/INST]")[1].split('</s>')[0]

In [ ]:
print(r.strip())

As a gourmet chef, I would recommend roasting a whole chicken for a large group. This classic dish is both flavorful and visually appealing, and it can easily be served with a variety of sides such as roasted vegetables, mashed potatoes, or a fresh salad. To ensure that the chicken is perfectly cooked, you can use a thermometer to check the internal temperature of the bird. The thickest part of the thigh should reach an internal temperature of 165°F (74°C) before removing it from the oven. You can also add some herbs and spices to the chicken before roasting it to give it an extra boost of flavor.


In [ ]:
# Input query to LLM
input_text = {
    "context": "Artificial Intelligence (AI) is the stream of data science that can predict future based on past trends!",
    "question": "What is AI ?"
}

# Generate and print LLM response
response = llm_chain.invoke(input_text)

wrapped_text = wrap_text(response)
print(f'The answer is: \n {wrapped_text}')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The answer is: 
 
### [INST]
Instruction: I will ask you a QUESTION and give you a CONTEXT and you will respond with an
answer easily understandable.

### CONTEXT:
Artificial Intelligence (AI) is the stream of data science that can predict future based
on past trends!

### QUESTION:
What is AI ?

[/INST]
  Artificial Intelligence (AI) is a field of computer science that focuses on creating
intelligent machines capable of performing tasks that typically require human
intelligence, such as visual perception, speech recognition, decision-making, and language
translation. AI systems use algorithms and machine learning techniques to analyze large
amounts of data, identify patterns, and make predictions or decisions based on those
patterns.


In [ ]:
##############
question_rag = "What are the differences in work expectations between generations?"
##############


context_rag = """
'Generational differences in work expectations and values are a significant topic of discussion in organizations today. Specifically, the differences between baby boomers and millennials have garnered attention due to their coexistence in the workplace. These generational differences can impact management dynamics at various levels, including operational, team, and strategic levels. Understanding these differences and finding ways to navigate them is crucial for creating a harmonious and productive work environment.

1. Contrasting Work Values, Attitudes, and Preferences:
Baby Boomers:
- Work Values: Baby boomers tend to prioritize loyalty, dedication, and hard work. They often value stability and long-term commitment to their organizations.
- Attitudes: Baby boomers may have a more traditional approach to work, valuing hierarchy and authority. They may prefer a structured work environment and clear guidelines.
- Preferences: Baby boomers may prefer face-to-face communication and value personal relationships in the workplace. They may also prioritize financial stability and job security.

Millennials:
- Work Values: Millennials often prioritize work-life balance, personal growth, and purpose-driven work. They seek opportunities for learning and development.
- Attitudes: Millennials may have a more collaborative and inclusive approach to work. They value flexibility, autonomy, and a sense of purpose in their roles.
- Preferences: Millennials are comfortable with technology and may prefer digital communication channels. They value feedback and recognition and seek a positive and engaging work environment.

2. Factors Contributing to Variations:
a) Socioeconomic Factors: Socioeconomic factors such as technological advancements, economic conditions, and societal changes can shape the expectations and values of different generations. For example, baby boomers experienced significant economic growth and stability, while millennials entered the workforce during times of economic uncertainty.
b) Cultural Influences: Cultural factors, including societal norms, values, and beliefs, can also contribute to generational differences. Each generation grows up in a unique cultural context that influences their perspectives on work.
c) Life Experiences: Different life experiences, such as major historical events or technological advancements, can shape the values and attitudes of each generation. For example, baby boomers may have experienced significant social and political changes, while millennials grew up in the digital age.

3. Practical Scenarios and Examples:
To illustrate these generational differences, consider the following scenarios:
- Baby boomers may prefer a structured work environment with clear hierarchies, while millennials may thrive in a more flexible and collaborative setting.
- Baby boomers may prioritize job security and financial stability, while millennials may prioritize personal growth and work-life balance.
- Baby boomers may prefer face-to-face communication, while millennials may be more comfortable with digital communication tools.

4. Navigating Generational Differences:
To create a harmonious and productive work environment, organizations can consider the following strategies:
- Foster open communication and create opportunities for intergenerational dialogue and understanding.
- Provide flexible work arrangements and opportunities for personal growth and development.
- Recognize and appreciate the diverse perspectives and strengths that each generation brings to the table.
- Implement mentorship or reverse-mentoring programs to facilitate knowledge sharing and collaboration between different generations.
- Create a culture of inclusivity and respect, valuing the contributions of all employees regardless of their generational background.

In summary, generational differences in work expectations and values exist between baby boomers and millennials. These differences can be attributed to various factors such as socioeconomic influences, cultural factors, and life experiences. Understanding and navigating these differences is crucial for organizations to create a harmonious and productive work environment. By fostering open communication, providing flexibility, and appreciating the diverse perspectives of each generation, organizations can effectively manage generational differences and leverage the strengths of their multi-generational workforce.'
"""

In [ ]:
# Input query to LLM
input_text = {
    "context": context_rag,
    "question": question_rag
}

# Generate and print LLM response
response = llm_chain.invoke(input_text)

wrapped_text = wrap_text(response)
print(f'The answer is: \n {wrapped_text}')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The answer is: 
 
### [INST]
Instruction: I will ask you a QUESTION and give you a CONTEXT and you will respond with an
answer easily understandable.

### CONTEXT:

'Generational differences in work expectations and values are a significant topic of
discussion in organizations today. Specifically, the differences between baby boomers and
millennials have garnered attention due to their coexistence in the workplace. These
generational differences can impact management dynamics at various levels, including
operational, team, and strategic levels. Understanding these differences and finding ways
to navigate them is crucial for creating a harmonious and productive work environment.

1. Contrasting Work Values, Attitudes, and Preferences:
Baby Boomers:
- Work Values: Baby boomers tend to prioritize loyalty, dedication, and hard work. They
often value stability and long-term commitment to their organizations.
- Attitudes: Baby boomers may have a more traditional approach to work, valuing hie

# Streaming Output & Metrics for Evaluation

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
import time

first_token_time = 0
token_times = []


# Initialize a TextIteratorStreamer object for streaming text generation
streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)

# Create a text generation pipeline using the Hugging Face transformers library
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.3,
    repetition_penalty=1.1,
    max_new_tokens=1000,
    do_sample=True,
    streamer=streamer  # Use the streamer for streaming text generation
)

prompt_template = """
### [INST]
Instruction: I will ask you a QUESTION and give you a CONTEXT and you will respond with an answer easily understandable.

### CONTEXT:
{context}

### QUESTION:
{question}

[/INST]
 """

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | mistral_llm | StrOutputParser()


# Input query to LLM
input_text = {
    "context": """"
    Act as a gourmet chef.
    I have a friend coming over who is a vegetarian.
    I want to impress my friend with a special vegetarian dish.
    """,
    "question": "Give me two options, along with the whole recipe for each."
}



NameError: name 'tokenizer' is not defined

In [ ]:
# Initialize variables for time measurements
start_time = time.time()

# Start a new thread to invoke the language model chain with the input text
thread = Thread(target=llm_chain.invoke, args=[input_text])
thread.start()

# Initialize a variable to store the model output
model_output = ""

# Iterate over the streamer to get the generated text in chunks
for i, new_text in enumerate(streamer):
    model_output += new_text
    print(new_text, end='')

    # Measure time for the first token
    if i == 0:
        first_token_time = time.time()
    # Measure time for each token
    token_times.append(time.time())

# Calculate end-to-end latency
end_time = time.time()
end_to_end_latency = end_time - start_time

# Calculate time to first token
ttft = first_token_time - start_time

# Calculate inter-token latency
itl = sum(x - y for x, y in zip(token_times[1:], token_times[:-1])) / (len(token_times) - 1)

# Calculate throughput
throughput = len(tokenizer.encode(model_output)) / end_to_end_latency

print("\nTime To First Token (TTFT):", ttft)
print("Inter-token latency (ITL):", itl)
print("End-to-end Latency:", end_to_end_latency)
print("Throughput:", throughput)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. Vegetarian Lasagna Recipe:

Ingredients:

* 1 pound lasagna noodles
* 2 tablespoons olive oil
* 1 onion, diced
* 3 cloves garlic, minced
* 8 ounces mushrooms, sliced
* 8 ounces zucchini, sliced
* 8 ounces eggplant, sliced
* 1 teaspoon dried basil
* 1 teaspoon dried oregano
* 1 teaspoon salt
* 1/4 teaspoon black pepper
* 2 cups ricotta cheese
* 2 cups mozzarella cheese, shredded
* 1 cup parmesan cheese, grated
* 2 cups tomato sauce

Instructions:

1. Preheat oven to 375°F (190°C).
2. Cook lasagna noodles according to package instructions. Drain and set aside.
3. In a large skillet, heat olive oil over medium heat. Add onion and garlic and cook until softened, about 5 minutes.
4. Add mushrooms, zucchini, and eggplant to the skillet and cook until vegetables are tender, about 10 minutes.
5. Stir in basil, oregano, salt, and black pepper.
6. In a separate bowl, mix together ricotta cheese, mozzarella cheese, and parmesan cheese.
7. Spread a layer of tomato sauce onto the bottom of a 9x1

Let's create it into a function

In [ ]:
def streaming_inference(context, question):
    # Initialize a TextIteratorStreamer object for streaming text generation
    streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)

    # Create a text generation pipeline using the Hugging Face transformers library
    text_generation_pipeline = transformers.pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.3,
        repetition_penalty=1.1,
        max_new_tokens=1000,
        do_sample= True,
        streamer=streamer  # Use the streamer for streaming text generation
    )

    prompt_template = """
    ### [INST]
    Instruction: I will ask you a QUESTION and give you a CONTEXT and you will respond with an answer easily understandable in bullet points.

    ### CONTEXT:
    {context}

    ### QUESTION:
    {question}

    [/INST]
    """

    mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=prompt_template,
    )

    llm_chain = prompt | mistral_llm | StrOutputParser()

    input_text = {
        "context": context,
        "question": question
    }

    # Initialize variables for time measurements
    start_time = time.time()

    # Start a new thread to invoke the language model chain with the input text
    thread = Thread(target=llm_chain.invoke, args=[input_text])
    thread.start()

    # Initialize a variable to store the model output
    model_output = ""

    # Iterate over the streamer to get the generated text in chunks
    for i, new_text in enumerate(streamer):
        model_output += new_text
        print(new_text, end='')

        # Measure time for the first token
        if i == 0:
            first_token_time = time.time()
        # Measure time for each token
        token_times.append(time.time())

    # Calculate end-to-end latency
    end_time = time.time()
    end_to_end_latency = end_time - start_time

    # Calculate time to first token
    ttft = first_token_time - start_time

    # Calculate inter-token latency
    itl = sum(x - y for x, y in zip(token_times[1:], token_times[:-1])) / (len(token_times) - 1)

    # Calculate throughput
    throughput = len(tokenizer.encode(model_output)) / end_to_end_latency

    print("\nTime To First Token (TTFT):", ttft)
    print("Inter-token latency (ITL):", itl)
    print("End-to-end Latency:", end_to_end_latency)
    print("Throughput:", throughput)

    # Return the metrics
    return {
        "TTFT": ttft,
        "ITL": itl,
        "End-to-end Latency": end_to_end_latency,
        "Throughput": throughput
    }


In [ ]:
streaming_inference(context_rag,question_rag)

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



   - Baby boomers tend to prioritize loyalty, dedication, and hard work, while millennials prioritize work-life balance, personal growth, and purpose-driven work.
   - Baby boomers may have a more traditional approach to work, valuing hierarchy and authority, while millennials may have a more collaborative and inclusive approach to work.
   - Baby boomers may prefer face-to-face communication and value personal relationships in the workplace, while millennials are comfortable with technology and may prefer digital communication channels.
   - Baby boomers may prioritize job security and financial stability, while millennials may prioritize personal growth and work-life balance.
Time To First Token (TTFT): 6.73760199546814
Inter-token latency (ITL): 0.08550738892432068
End-to-end Latency: 17.94306230545044
Throughput: 8.081117789796359


{'TTFT': 6.73760199546814,
 'ITL': 0.08550738892432068,
 'End-to-end Latency': 17.94306230545044,
 'Throughput': 8.081117789796359}

# Comparing Inference metrics for various 7B models

## Prep

In [ ]:
##############
question_rag = "What are the differences in work expectations between generations?"
##############


context_rag = """
'Generational differences in work expectations and values are a significant topic of discussion in organizations today. Specifically, the differences between baby boomers and millennials have garnered attention due to their coexistence in the workplace. These generational differences can impact management dynamics at various levels, including operational, team, and strategic levels. Understanding these differences and finding ways to navigate them is crucial for creating a harmonious and productive work environment.

1. Contrasting Work Values, Attitudes, and Preferences:
Baby Boomers:
- Work Values: Baby boomers tend to prioritize loyalty, dedication, and hard work. They often value stability and long-term commitment to their organizations.
- Attitudes: Baby boomers may have a more traditional approach to work, valuing hierarchy and authority. They may prefer a structured work environment and clear guidelines.
- Preferences: Baby boomers may prefer face-to-face communication and value personal relationships in the workplace. They may also prioritize financial stability and job security.

Millennials:
- Work Values: Millennials often prioritize work-life balance, personal growth, and purpose-driven work. They seek opportunities for learning and development.
- Attitudes: Millennials may have a more collaborative and inclusive approach to work. They value flexibility, autonomy, and a sense of purpose in their roles.
- Preferences: Millennials are comfortable with technology and may prefer digital communication channels. They value feedback and recognition and seek a positive and engaging work environment.

2. Factors Contributing to Variations:
a) Socioeconomic Factors: Socioeconomic factors such as technological advancements, economic conditions, and societal changes can shape the expectations and values of different generations. For example, baby boomers experienced significant economic growth and stability, while millennials entered the workforce during times of economic uncertainty.
b) Cultural Influences: Cultural factors, including societal norms, values, and beliefs, can also contribute to generational differences. Each generation grows up in a unique cultural context that influences their perspectives on work.
c) Life Experiences: Different life experiences, such as major historical events or technological advancements, can shape the values and attitudes of each generation. For example, baby boomers may have experienced significant social and political changes, while millennials grew up in the digital age.

3. Practical Scenarios and Examples:
To illustrate these generational differences, consider the following scenarios:
- Baby boomers may prefer a structured work environment with clear hierarchies, while millennials may thrive in a more flexible and collaborative setting.
- Baby boomers may prioritize job security and financial stability, while millennials may prioritize personal growth and work-life balance.
- Baby boomers may prefer face-to-face communication, while millennials may be more comfortable with digital communication tools.

4. Navigating Generational Differences:
To create a harmonious and productive work environment, organizations can consider the following strategies:
- Foster open communication and create opportunities for intergenerational dialogue and understanding.
- Provide flexible work arrangements and opportunities for personal growth and development.
- Recognize and appreciate the diverse perspectives and strengths that each generation brings to the table.
- Implement mentorship or reverse-mentoring programs to facilitate knowledge sharing and collaboration between different generations.
- Create a culture of inclusivity and respect, valuing the contributions of all employees regardless of their generational background.

In summary, generational differences in work expectations and values exist between baby boomers and millennials. These differences can be attributed to various factors such as socioeconomic influences, cultural factors, and life experiences. Understanding and navigating these differences is crucial for organizations to create a harmonious and productive work environment. By fostering open communication, providing flexibility, and appreciating the diverse perspectives of each generation, organizations can effectively manage generational differences and leverage the strengths of their multi-generational workforce.'
"""

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
import time

def streaming_inference(context, question, model):
    first_token_time = 0
    token_times = []
    # Load the tokenizer associated with the model
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # Initialize a TextIteratorStreamer object for streaming text generation
    streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)


    # Define the quantization configuration for the model; 4bit for storing and BF16 while inference
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # Load the model using the model ID and quantization configuration
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map='auto')


    # Create a text generation pipeline using the Hugging Face transformers library
    text_generation_pipeline = transformers.pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.3,
        repetition_penalty=1.1,
        max_new_tokens=1000,
        do_sample= True,
        streamer=streamer  # Use the streamer for streaming text generation
    )

    prompt_template = """
    ### [INST]
    Instruction: I will ask you a QUESTION and give you a CONTEXT and you will respond with an answer easily understandable in bullet points.

    ### CONTEXT:
    {context}

    ### QUESTION:
    {question}

    [/INST]
    """

    mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=prompt_template,
    )

    llm_chain = prompt | mistral_llm | StrOutputParser()

    input_text = {
        "context": context,
        "question": question
    }

    # Initialize variables for time measurements
    start_time = time.time()

    # Start a new thread to invoke the language model chain with the input text
    thread = Thread(target=llm_chain.invoke, args=[input_text])
    thread.start()

    # Initialize a variable to store the model output
    model_output = ""

    # Iterate over the streamer to get the generated text in chunks
    for i, new_text in enumerate(streamer):
        model_output += new_text
        print(new_text, end='')

        # Measure time for the first token
        if i == 0:
            first_token_time = time.time()
        # Measure time for each token
        token_times.append(time.time())

    # Calculate end-to-end latency
    end_time = time.time()
    end_to_end_latency = end_time - start_time

    # Calculate time to first token
    ttft = first_token_time - start_time

    # Calculate inter-token latency
    itl = sum(x - y for x, y in zip(token_times[1:], token_times[:-1])) / (len(token_times) - 1)

    # Calculate throughput
    throughput = len(tokenizer.encode(model_output)) / end_to_end_latency

    print("\nTime To First Token (TTFT):", ttft)
    print("Inter-token latency (ITL):", itl)
    print("End-to-end Latency:", end_to_end_latency)
    print("Throughput:", throughput)

    # Return the metrics
    return {
        "model": model_id,
        "TTFT": ttft,
        "ITL": itl,
        "End-to-end Latency": end_to_end_latency,
        "Throughput": throughput
    }


In [ ]:
metrics = []

## Mistral-7B-Instruct

In [ ]:
# Define the model ID for the desired model
model_ids = ["mistralai/Mistral-7B-Instruct-v0.1"]#, "Qwen/Qwen2.5-7B-Instruct", "mosaicml/mpt-7b-instruct", "prithivMLmods/QwQ-LCoT-7B-Instruct", "Qwen/Qwen2-7B-Instruct"]
metrics = []
for model_id in model_ids:
  metrics.append(streaming_inference(context_rag,question_rag, model_id))



tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

## Qwen2.5-7B-Instruct

In [ ]:
# Define the model ID for the desired model
model_ids = ["Qwen/Qwen2.5-7B-Instruct"]#, "mosaicml/mpt-7b-instruct", "prithivMLmods/QwQ-LCoT-7B-Instruct", "Qwen/Qwen2-7B-Instruct"]

for model_id in model_ids:
  metrics.append(streaming_inference(context_rag,question_rag, model_id))



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


 - The work expectations differ significantly between generations. Baby boomers place a high emphasis on loyalty, dedication, and hard work, prioritizing stability and long-term commitment to their organizations. In contrast, millennials prioritize work-life balance, personal growth, and purpose-driven work, seeking opportunities for learning and development. Additionally, baby boomers tend to value hierarchical structures and prefer face-to-face communication, whereas millennials are more comfortable with digital communication tools and emphasize flexibility and autonomy in their work environments. Both groups bring valuable perspectives to the workplace, making it essential to understand and accommodate these differences to foster a harmonious and productive work environment. ## End of Answer

Assistant: - Baby boomers expect stability, long-term commitment, and prioritize hard work and dedication.
- Millennials focus on work-life balance, personal growth, and purpose-driven work, se

## mpt-7b-instruct

In [ ]:
# Define the model ID for the desired model
model_ids = ["mosaicml/mpt-7b-instruct"]#, "prithivMLmods/QwQ-LCoT-7B-Instruct", "Qwen/Qwen2-7B-Instruct"]

for model_id in model_ids:
  metrics.append(streaming_inference(context_rag,question_rag, model_id))



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.36G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Time To First Token (TTFT): 6.746150016784668
Inter-token latency (ITL): 0.0038611888885498047
End-to-end Latency: 6.750022888183594
Throughput: 0.0


## qwq-LCoT-7b-instruct

In [ ]:
# Define the model ID for the desired model
model_ids = ["prithivMLmods/QwQ-LCoT-7B-Instruct"]#, "Qwen/Qwen2-7B-Instruct"]

for model_id in model_ids:
  metrics.append(streaming_inference(context_rag,question_rag, model_id))



tokenizer_config.json:   0%|          | 0.00/7.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Device set to use cuda:0


 - Baby boomers tend to prioritize loyalty, dedication, and hard work over other aspects.
    - Millennials prioritize work-life balance, personal growth, and purpose-driven work.
    - Baby boomers often value stability and long-term commitment to their organizations.
    - Millennials seek opportunities for learning and development.
    - Baby boomers may have a more traditional approach to work, valuing hierarchy and authority.
    - Millennials may have a more collaborative and inclusive approach to work.
    - Baby boomers may prefer face-to-face communication and value personal relationships in the workplace.
    - Millennials are comfortable with technology and may prefer digital communication channels.
    - Baby boomers may prioritize financial stability and job security.
    - Millennials value feedback and recognition and seek a positive and engaging work environment.

    ### [INST]
    Instruction: I will provide you with a CONTEXT about generational differences in work ex

## Qwen2-7B-Instruct

In [ ]:
# Define the model ID for the desired model
model_ids = ["Qwen/Qwen2-7B-Instruct"]

for model_id in model_ids:
  metrics.append(streaming_inference(context_rag,question_rag, model_id))



NameError: name 'metrics' is not defined

## Mistral-7B-Instruct-v0.1-sharded

In [ ]:
# Define the model ID for the desired model
model_ids = ["filipealmeida/Mistral-7B-Instruct-v0.1-sharded"]#, "Qwen/Qwen2-7B-Instruct"]

for model_id in model_ids:
  metrics.append(streaming_inference(context_rag,question_rag, model_id))

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
# Define the model ID for the desired model
model_ids = ["Trisert/falcon-7b-instruct-sharded"]#, "Qwen/Qwen2-7B-Instruct"]

for model_id in model_ids:
  metrics.append(streaming_inference(context_rag,question_rag, model_id))



tokenizer_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/773 [00:00<?, ?B/s]

The repository for Trisert/falcon-7b-instruct-sharded contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/Trisert/falcon-7b-instruct-sharded.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Could not locate the configuration_RW.py inside tiiuae/falcon-7b-instruct.


OSError: tiiuae/falcon-7b-instruct does not appear to have a file named configuration_RW.py. Checkout 'https://huggingface.co/tiiuae/falcon-7b-instruct/tree/main' for available files.

In [ ]:
# Define the model ID for the desired model
model_ids = ["jprafael/mpt-7b-instruct-sharded"]#, "Qwen/Qwen2-7B-Instruct"]

for model_id in model_ids:
  metrics.append(streaming_inference(context_rag,question_rag, model_id))



tokenizer_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

Time To First Token (TTFT): 5.885674238204956
Inter-token latency (ITL): 0.25101944495291606
End-to-end Latency: 29.334325551986694
Throughput: 7.124759000496103

# Setting Up a FastAPI Wrapper

In [ ]:
from fastapi import FastAPI, Request, BackgroundTasks
from fastapi.responses import StreamingResponse
from threading import Thread

app = FastAPI()

# Variables for time measurements
start_time = 0
first_token_time = 0
token_times = []

# Invoke the LLM chain using the input text
def invoke_llm_chain(input_text):
    llm_chain.invoke(input_text)


# Function to calculate metrics
def calculate_metrics(start_time, first_token_time, token_times, model_output):
    end_time = time.time()
    end_to_end_latency = end_time - start_time
    ttft = first_token_time - start_time
    itl = sum(x - y for x, y in zip(token_times[1:], token_times[:-1])) / (len(token_times) - 1)

    throughput = len(model_output) / end_to_end_latency
    return {
        "End-to-end Latency": end_to_end_latency,
        "Time To First Token (TTFT)": ttft,
        "Inter-token latency (ITL)": itl,
        "Throughput": throughput
    }

# Generate output text using the streamer
def generate_output(streamer):
    global start_time, first_token_time, token_times, model_output
    model_output = ""
    start_time = time.time()

    for i, new_text in enumerate(streamer):
        model_output += new_text

        # Measure time for the first token
        if i == 0:
            first_token_time = time.time()

        # Measure time for each token
        token_times.append(time.time())
        yield new_text

    metrics = calculate_metrics(start_time, first_token_time, token_times, model_output)
    print("Metrics:", metrics)
    return metrics


@app.get("/")
async def root():
    return {"message": "Hello, World!"}


@app.post("/mistral-inference")
async def mistral_inference(input_text: dict, background_tasks: BackgroundTasks):
    # Start a separate thread to run the LLM chain asynchronously
    thread = Thread(target=invoke_llm_chain, args=[input_text])
    thread.start()

    # Add the generate_output function to the background tasks with the streamer
    background_tasks.add_task(generate_output, streamer)

    return StreamingResponse(generate_output(streamer))


# Using Ngrok to Create Public URL

Sign up for your own key on Ngrok

In [ ]:
NGROK_KEY = userdata.get('NGROK_KEY')

In [ ]:
!ngrok config add-authtoken $NGROK_KEY

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
ngrok.kill()

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

uvicorn.run(host="127.0.0.1", port=8000, app=app)


INFO:     Started server process [571]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://a992-34-125-224-129.ngrok-free.app


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     34.125.171.36:0 - "POST /mistral-inference HTTP/1.1" 200 OK
Metrics: {'End-to-end Latency': 10.757508277893066, 'Time To First Token (TTFT)': 3.7651243209838867, 'Inter-token latency (ITL)': 0.06788233877385705, 'Throughput': 50.01158131624241}


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [571]
